In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from google.colab import drive

drive.mount('/content/drive')

SADpath="/content/drive/MyDrive/SAD_v1.csv"
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # For 3 stress levels
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

print("Datasets tokenized successfully.")

Map:   0%|          | 0/5480 [00:00<?, ? examples/s]

Map:   0%|          | 0/1370 [00:00<?, ? examples/s]

Datasets tokenized successfully.


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, classification_report


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none'
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
)


trainer.train()


predictions = trainer.predict(tokenized_eval_dataset)


eval_labels = tokenized_eval_dataset['labels']


accuracy = accuracy_score(eval_labels, predictions.predictions.argmax(axis=-1))


print("Accuracy: ", accuracy)


print("Classification Report: ")
print(classification_report(eval_labels, predictions.predictions.argmax(axis=-1)))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/5480 [00:00<?, ? examples/s]

Map:   0%|          | 0/1370 [00:00<?, ? examples/s]

Step,Training Loss
10,0.834300
20,0.770500
30,0.688300
40,0.557500
50,0.473200
60,0.387200
70,0.299400
80,0.308000
90,0.181200
100,0.197900


Accuracy:  0.9474452554744526
Classification Report: 
              precision    recall  f1-score   support

           0       0.54      0.25      0.35        75
           1       0.96      0.99      0.97      1295

    accuracy                           0.95      1370
   macro avg       0.75      0.62      0.66      1370
weighted avg       0.94      0.95      0.94      1370



In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from google.colab import drive

drive.mount('/content/drive')

SADpath="/content/drive/MyDrive/SAD_v1.csv"
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
import os

save_path = "/content/drive/MyDrive/fine_tuned_bert_model"
os.makedirs(save_path, exist_ok=True)

trainer.save_model(save_path)

tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to: {save_path}")

Model and tokenizer saved to: /content/drive/MyDrive/fine_tuned_bert_model


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

save_path = "/content/drive/MyDrive/bert_merged_model"


loaded_model = BertForSequenceClassification.from_pretrained(save_path)


loaded_tokenizer = BertTokenizer.from_pretrained(save_path)

print("Model and tokenizer loaded successfully.")

Model and tokenizer loaded successfully.


In [ ]:
def tokenize_function(examples):
    return loaded_tokenizer(examples['sentence'], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

print("Datasets tokenized successfully.")

NameError: name 'train_dataset' is not defined

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report


training_args = TrainingArguments(
    output_dir='./results',
    report_to='none'
)


trainer = Trainer(
    model=loaded_model,
    args=training_args,
    eval_dataset=tokenized_eval_dataset
)


predictions = trainer.predict(tokenized_eval_dataset)


preds = predictions.predictions.argmax(axis=-1);


eval_labels = tokenized_eval_dataset['labels']


accuracy = accuracy_score(eval_labels, preds)


print(f"Accuracy: {accuracy}")


print("Classification Report: ")
print(classification_report(eval_labels, preds))

Accuracy: 0.9445255474452555
Classification Report: 
              precision    recall  f1-score   support

           0       0.49      0.23      0.31        75
           1       0.96      0.99      0.97      1295

    accuracy                           0.94      1370
   macro avg       0.72      0.61      0.64      1370
weighted avg       0.93      0.94      0.93      1370



In [ ]:
import pandas as pd


comparison_df = pd.DataFrame({
    'text': eval_dataset['sentence'],
    'true_label': eval_dataset['labels'],
    'predicted_label': preds
})


print(comparison_df.head(10))

                                                text  true_label  \
0  i have a really busy schedule this weekend and...           1   
1  our president is sick and he needs to be remov...           1   
2  I'm feeling so lost in life is my biggest prob...           1   
3                      we are short staffed at work.           1   
4  Late, mind you, because someone messed up my s...           0   
5  i got insanely sick somewhere around wednesday...           1   
6  been trying for days to book doc apt and still...           1   
7                                  Looking for a Job           1   
8                              sick kids and animals           1   
9            Got into another fight with my brother.           1   

   predicted_label  
0                1  
1                1  
2                1  
3                1  
4                1  
5                1  
6                1  
7                1  
8                1  
9                1  


In [ ]:
pip install soundfile librosa pydub transformers

# Load bert model

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

save_path = "/content/drive/MyDrive/bert_merged_model"


loaded_model = BertForSequenceClassification.from_pretrained(save_path);


loaded_tokenizer = BertTokenizer.from_pretrained(save_path);



## Load ASR Model ( from facebook)


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


model_name = "facebook/wav2vec2-base-960h"
asr_processor = Wav2Vec2Processor.from_pretrained(model_name);
asr_model = Wav2Vec2ForCTC.from_pretrained(model_name);

print(f"ASR model '{model_name}' and processor loaded successfully.")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ASR model 'facebook/wav2vec2-base-960h' and processor loaded successfully.


# One item prediction

In [ ]:
import librosa
from pydub import AudioSegment
import io

def convert_mp3_to_wav_and_load(mp3_path, target_sample_rate=16000):

    audio = AudioSegment.from_mp3(mp3_path);


    wav_data = io.BytesIO();
    audio.export(wav_data, format="wav");
    wav_data.seek(0);


    y, sr = librosa.load(wav_data, sr=target_sample_rate);
    return y, sr;



In [ ]:
import torch
import numpy as np

def transcribe_audio(mp3_path):
    """
    Transcribes an MP3 audio file to text using the loaded ASR model.
    """

    waveform, sample_rate = convert_mp3_to_wav_and_load(mp3_path, target_sample_rate=16000);


    input_values = asr_processor(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values;


    with torch.no_grad():
        logits = asr_model(input_values).logits;

    predicted_ids = torch.argmax(logits, dim=-1);
    transcription = asr_processor.batch_decode(predicted_ids)[0];
    return transcription;



In [ ]:
import numpy as np
from pydub import AudioSegment
import os
import torch

def predict_stress(text):
    """
    Predicts the stress label (0 or 1) for a given text using the loaded BERT model,
    applying label inversion as previously identified.
    """

    inputs = loaded_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128);


    device = loaded_model.device;
    inputs = {k: v.to(device) for k, v in inputs.items()};


    with torch.no_grad():
        outputs = loaded_model(**inputs);

    logits = outputs.logits;
    predicted_class_id = torch.argmax(logits, dim=-1).item();

    return predicted_class_id;

def process_audio_for_stress(mp3_file_path):
    """
    Orchestrates the ASR and BERT prediction steps for an MP3 file.
    Accepts an MP3 file path, transcribes it, and then outputs the predicted stress level.
    """

    transcribed_text = transcribe_audio(mp3_file_path);
    print(f"Transcribed Text: {transcribed_text}");


    predicted_stress_label = predict_stress(transcribed_text);

    return transcribed_text, predicted_stress_label;


sample_mp3_path = "/content/sample_audio.mp3";


duration_ms = 3000;
sample_rate = 16000;
silence_audio = AudioSegment.silent(duration=duration_ms, frame_rate=sample_rate);
silence_audio.export(sample_mp3_path, format="mp3");

print(f"Dummy MP3 created at: {sample_mp3_path}");


transcribed_text, predicted_stress_label = process_audio_for_stress(sample_mp3_path);

print(f"Final Transcribed Text: '{transcribed_text}'");
print(f"Final Predicted Stress Label: {predicted_stress_label} (0: No Stress, 1: Stress)");

Dummy MP3 created at: /content/sample_audio.mp3
Transcribed Text: 
Final Transcribed Text: ''
Final Predicted Stress Label: 0 (0: No Stress, 1: Stress)


In [ ]:
import os


base_mp3_path = "/content/drive/MyDrive/WuHaoAllenCentad/stress_id_wav_filtered";


mp3_files = [
    "2ea4_Counting1.mp3",
    "45lx_Speaking.mp3"
];

print("Processing audio files for transcription and stress identification...");
print("-------------------------------------------------------------------");

results = [];

for file_name in mp3_files:
    mp3_full_path = os.path.join(base_mp3_path, file_name);
    print(f"\nProcessing file: {file_name}");

    if not os.path.exists(mp3_full_path):
        print(f"ERROR: File not found at {mp3_full_path}. Please ensure the MP3 file is uploaded to your Google Drive.");
        transcribed_text = "N/A (File Not Found)";
        predicted_stress_label = "N/A";
    else:
        try:
            transcribed_text, predicted_stress_label = process_audio_for_stress(mp3_full_path);
        except Exception as e:
            print(f"ERROR processing {file_name}: {e}");
            transcribed_text = "N/A (Processing Error)";
            predicted_stress_label = "N/A";

    results.append({
        "filename": file_name,
        "transcription": transcribed_text,
        "stress_label": predicted_stress_label
    });

print("-------------------------------------------------------------------");
print("Processing complete. Here are the results:");
for res in results:
    print(f"\nFile: {res['filename']}");
    print(f"  Transcribed Text: '{res['transcription']}'");
    print(f"  Predicted Stress Label: {res['stress_label']} (0: No Stress, 1: Stress)");

Processing audio files for transcription and stress identification...
-------------------------------------------------------------------

Processing file: 2ea4_Counting1.mp3
Transcribed Text: ONE HUNDRED NINETY SEVEN NINETY FOUR NINETY ONE EIGHTY EIGHT A EIGHTY FIVE EIGHTY TWO SEVENTY NINE SEVENTY SIX SEVENTY THREE SEVEN SIXTY SEVEN SIXTY FOUR SIXTY ONE FIFTY EIGHT FIFTY FIVE FIFTY TWO FORTY NINE FORTY SIX FORTY THREE FOUR THIRTY SEVEN THIRTEIGHT FOUR

Processing file: 45lx_Speaking.mp3
Transcribed Text: OND O CA SO I'M BELY A SHET WITER PEOPLE AND EA I CAN SOD B BROES AND ERI AN GLE SHOCKINGT PEOPLE BUT HESNT LIOY AND  WHAT AS I I CAN WRITE WELL I CAN SOLD MOHE MONEY A TROS AND YE SANT TOLK TO LONGINGE AND I AM GOING TO PLY IN ANIN EXPERIMENTS AN MY WORK SO IF IT WOST  WODWO TENTERS THER LIS AD NOT VEY GOOD TO IMAGINE PEOPLE AND  SON TAAN SO AN  SLOY I KNOW THAT YOU GOT TRO FINKS BETTER YOU WORKIN DO BETTER SHALLY GO EE HAYHIN  NOTE A WATY NOISE ERI IS NETE
--------------------------

# Testing BERT Model on complex sentences

In [ ]:
import torch
import torch.nn.functional as F

def predict_text_stress(text):
    """
    Predicts whether a given text indicates stress using the fine-tuned BERT model.
    Args:
        text (str): The input text to analyze.
    Returns:
        str: The predicted label ("Stressed" or "Not Stressed").
        float: The confidence score of the prediction.
    """

    model.eval();


    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128);


    device = model.device;
    inputs = {k: v.to(device) for k, v in inputs.items()};


    with torch.no_grad():
        outputs = model(**inputs);


    logits = outputs.logits;
    probs = F.softmax(logits, dim=-1);


    pred_class = torch.argmax(probs, dim=-1).item();
    confidence = probs[0][pred_class].item();


    label_map = {0: "Not Stressed", 1: "Stressed"};

    return label_map[pred_class], confidence;


print("Text-only Stress Identification Pipeline Ready.\n");

test_sentences = [
    "I am feeling really overwhelmed with all the work I have to do.",
    "The weather is beautiful today, and I feel great.",
    "I'm so anxious about the upcoming deadline, I can't sleep.",
    "I just had a nice lunch with my friends.",
    "I can't sleep because its exams season. I can't fail biology again else I'm done for",
    "I have an exam tomorrow but its chill. Probably not that hard since its just an aptitude test",
    "I am feeling anxious. Cooking is such a relaxing activity, because its just fun to do! Weather",
    "Testing testing testing"
];

for sentence in test_sentences:
    label, conf = predict_text_stress(sentence);
    print(f"Text: \"{sentence}\"");
    print(f"Prediction: {label} (Confidence: {conf:.2%})\n");

Text-only Stress Identification Pipeline Ready.

Text: "I am feeling really overwhelmed with all the work I have to do."
Prediction: Stressed (Confidence: 99.98%)

Text: "The weather is beautiful today, and I feel great."
Prediction: Not Stressed (Confidence: 99.70%)

Text: "I'm so anxious about the upcoming deadline, I can't sleep."
Prediction: Stressed (Confidence: 99.98%)

Text: "I just had a nice lunch with my friends."
Prediction: Stressed (Confidence: 93.98%)

Text: "I can't sleep because its exams season. I can't fail biology again else I'm done for"
Prediction: Stressed (Confidence: 99.90%)

Text: "I have an exam tomorrow but its chill. Probably not that hard since its just an aptitude test"
Prediction: Stressed (Confidence: 99.98%)

Text: "I am feeling anxious. Cooking is such a relaxing activity, because its just fun to do! Weather"
Prediction: Stressed (Confidence: 81.13%)

Text: "Testing testing testing"
Prediction: Stressed (Confidence: 98.11%)



## Training BERT Model : Load and merge SAD and dreaddit into one

In [ ]:
import pandas as pd
import os


sad_path = '/content/drive/MyDrive/SAD_v1.csv';
dreaddit_train_path = '/content/dreaddit-train.csv';
dreaddit_test_path = '/content/dreaddit-test.csv';


df_sad = pd.read_csv(sad_path);
print(f"SAD dataset loaded. Shape: {df_sad.shape}");

df_dreaddit_train = pd.read_csv(dreaddit_train_path);
print(f"Dreaddit Train dataset loaded. Shape: {df_dreaddit_train.shape}");


if os.path.exists(dreaddit_test_path):
    df_dreaddit_test = pd.read_csv(dreaddit_test_path);
    print(f"Dreaddit Test dataset loaded. Shape: {df_dreaddit_test.shape}");
else:
    print(f"Warning: {dreaddit_test_path} not found. Proceeding without it.");
    df_dreaddit_test = pd.DataFrame(columns=['text', 'label']);


df_sad = df_sad.rename(columns={'sentence': 'text', 'is_stressor': 'labels'});

df_sad = df_sad[['text', 'labels']];


if 'label' in df_dreaddit_train.columns:
    df_dreaddit_train = df_dreaddit_train.rename(columns={'label': 'labels'});
df_dreaddit_train = df_dreaddit_train[['text', 'labels']];


if 'label' in df_dreaddit_test.columns:
    df_dreaddit_test = df_dreaddit_test.rename(columns={'label': 'labels'});
if not df_dreaddit_test.empty:
    df_dreaddit_test = df_dreaddit_test[['text', 'labels']];


dfs_to_merge = [df_sad, df_dreaddit_train];
if not df_dreaddit_test.empty:
    dfs_to_merge.append(df_dreaddit_test);

merged_df = pd.concat(dfs_to_merge, ignore_index=True);


print(f"Merged DataFrame shape: {merged_df.shape}");
print(merged_df.head());

SAD dataset loaded. Shape: (6850, 24)
Dreaddit Train dataset loaded. Shape: (2838, 116)
Merged DataFrame shape: (9688, 2)
                                                text  labels
0                                                  ?       0
1                        2 of my foster kittens died       1
2  A better question would be what is not stressi...       1
3                             a big project at work.       0
4  a certain person at work is being really angry...       1


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset


merged_df['labels'] = merged_df['labels'].astype(int);


train_df, eval_df = train_test_split(merged_df, test_size=0.2, random_state=42, stratify=merged_df['labels']);


train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True));
eval_dataset = Dataset.from_pandas(eval_df.reset_index(drop=True));

print("Data preprocessing complete.");
print(f"Train dataset size: {len(train_dataset)}");
print(f"Eval dataset size: {len(eval_dataset)}");

Data preprocessing complete.
Train dataset size: 7750
Eval dataset size: 1938


In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased');

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True);


tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True);
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True);

print("Tokenization complete.");
print("Keys in tokenized dataset:", tokenized_train_dataset[0].keys());

Map:   0%|          | 0/7750 [00:00<?, ? examples/s]

Map:   0%|          | 0/1938 [00:00<?, ? examples/s]

Tokenization complete.
Keys in tokenized dataset: dict_keys(['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2);

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
);

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset
);

print("Starting training...");
trainer.train();
print("Training complete.");


predictions = trainer.predict(tokenized_eval_dataset);
preds = predictions.predictions.argmax(axis=-1);
true_labels = tokenized_eval_dataset['labels'];


accuracy = accuracy_score(true_labels, preds);
print(f"Accuracy: {accuracy}");
print("Classification Report:");
print(classification_report(true_labels, preds));

Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


{'loss': 0.3298, 'grad_norm': 4.193098068237305, 'learning_rate': 4.99e-05, 'epoch': 1.0309278350515463}
{'loss': 0.1806, 'grad_norm': 0.7137150168418884, 'learning_rate': 2.3874345549738222e-05, 'epoch': 2.0618556701030926}
{'train_runtime': 920.9339, 'train_samples_per_second': 25.246, 'train_steps_per_second': 1.58, 'train_loss': 0.19272077239256136, 'epoch': 3.0}
Training complete.
Accuracy: 0.913828689370485
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.72      0.75       345
           1       0.94      0.96      0.95      1593

    accuracy                           0.91      1938
   macro avg       0.86      0.84      0.85      1938
weighted avg       0.91      0.91      0.91      1938



# Save model

In [ ]:
import os
from transformers import BertForSequenceClassification, BertTokenizer

destination_path = '/content/drive/MyDrive/bert_merged_model';


model.save_pretrained(destination_path);
tokenizer.save_pretrained(destination_path);

print(f"Model and tokenizer successfully saved to '{destination_path}'.");

Model and tokenizer successfully loaded from '/content/drive/MyDrive/fine_tuned_bert_model' and saved to '/content/drive/MyDrive/bert_merged_model'.


## Evaluate Model

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, classification_report


training_args = TrainingArguments(
    output_dir='./results',
    report_to='none'
);


trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_eval_dataset
);


predictions = trainer.predict(tokenized_eval_dataset);
preds = predictions.predictions.argmax(axis=-1);
true_labels = tokenized_eval_dataset['labels'];
accuracy = accuracy_score(true_labels, preds);

print(f"Accuracy on merged dataaset: {accuracy}");
print(classification_report(true_labels, preds));

Accuracy on merged dataset: 0.913828689370485
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.72      0.75       345
           1       0.94      0.96      0.95      1593

    accuracy                           0.91      1938
   macro avg       0.86      0.84      0.85      1938
weighted avg       0.91      0.91      0.91      1938



In [ ]:
import torch
import torch.nn.functional as F
from pydub import AudioSegment
import os

def predict_stress(text):
    """
    Predicts the stress probability (class 1) for a given text using the loaded BERT model.
    """

    loaded_model.eval();


    inputs = loaded_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128);


    device = loaded_model.device;
    inputs = {k: v.to(device) for k, v in inputs.items()};


    with torch.no_grad():
        outputs = loaded_model(**inputs);

    logits = outputs.logits;

    probs = F.softmax(logits, dim=-1);


    stressed_probability = probs[0][1].item();
    return stressed_probability;

def process_audio_for_stress(mp3_file_path):
    """
    Orchestrates the ASR and BERT prediction steps for an MP3 file.
    Accepts an MP3 file path, transcribes it, and then outputs the predicted stress probability.
    """

    transcribed_text = transcribe_audio(mp3_file_path);
    print(f"Transcribed Text: {transcribed_text}");


    predicted_stress_probability = predict_stress(transcribed_text);

    return transcribed_text, predicted_stress_probability;


if 'sample_mp3_path' not in locals() or not os.path.exists(sample_mp3_path):
    sample_mp3_path = "/content/sample_audio.mp3";
    duration_ms = 3000;
    sample_rate = 16000;
    silence_audio = AudioSegment.silent(duration=duration_ms, frame_rate=sample_rate);
    silence_audio.export(sample_mp3_path, format="mp3");
    print(f"Dummy MP3 created at: {sample_mp3_path}");

print("Updated 'predict_stress' and 'process_audio_for_stress' functions.");


transcribed_text, predicted_stress_probability = process_audio_for_stress(sample_mp3_path);

print(f"Final Transcribed Text: '{transcribed_text}'");
print(f"Final Predicted Stress Probability (class 1): {predicted_stress_probability:.4f}");

Updated 'predict_stress' and 'process_audio_for_stress' functions.
Transcribed Text: 
Final Transcribed Text: ''
Final Predicted Stress Probability (class 1): 0.0847


## Evaluation on stressidtest2.csv + Get probabilities for all audio files in both eval and train for ensemble

In [ ]:
import librosa
from pydub import AudioSegment
import io

def load_audio_and_resample(file_path, target_sample_rate=16000):
    """
    Loads an audio file (MP3, WAV, etc.), resamples it to the target sample rate,
    and returns it as a NumPy array.
    """

    audio = AudioSegment.from_file(file_path);


    wav_data = io.BytesIO();
    audio.export(wav_data, format="wav");
    wav_data.seek(0);


    y, sr = librosa.load(wav_data, sr=target_sample_rate);
    return y, sr;

print("Function 'load_audio_and_resample' defined for general audio processing.");

def transcribe_audio(file_path):
    """
    Transcribes an audio file to text using the loaded ASR model.
    """

    waveform, sample_rate = load_audio_and_resample(file_path, target_sample_rate=16000);


    input_values = asr_processor(waveform, sampling_rate=sample_rate, return_tensors="pt").input_values;


    with torch.no_grad():
        logits = asr_model(input_values).logits;

    predicted_ids = torch.argmax(logits, dim=-1);
    transcription = asr_processor.batch_decode(predicted_ids)[0];
    return transcription;

print("Updated 'transcribe_audio' to use 'load_audio_and_resample'.");

Function 'load_audio_and_resample' defined for general audio processing.
Updated 'transcribe_audio' to use 'load_audio_and_resample'.


In [ ]:
import os
import pandas as pd

wav_files_base_path = "/content/drive/MyDrive/WuHaoAllenCentad/stress_id_wav_filtered"

all_files = os.listdir(wav_files_base_path)

wav_files = [f for f in all_files if f.endswith('.wav')]
results = []

for file_name in wav_files:
    full_file_path = os.path.join(wav_files_base_path, file_name)
    print(f" cur file: {file_name}")

    if not os.path.exists(full_file_path):
        print(f"not found {full_file_path}")
        transcribed_text = "N/A"
        predicted_stress_probability = "N/A"
    else:
        try:
            # process_audio_for_stress now returns probability
            transcribed_text, predicted_stress_probability = process_audio_for_stress(full_file_path)
            print(f"  Predicted Stress prob: {predicted_stress_probability:.4f}")
        except Exception as e:
            print(f"error {file_name}: {e}")
            transcribed_text = "N/A"
            predicted_stress_probability = "N/A"

    results.append({
        "filename": file_name,
        "transcription": transcribed_text,
        "stress_probability": predicted_stress_probability
    })

results_df = pd.DataFrame(results)
print("\nResults DataFrame:")
print(results_df)

In [ ]:
import pandas as pd
import os


results_df_cleaned = pd.DataFrame(results);
results_df_cleaned['subject/task'] = results_df_cleaned['filename'].apply(lambda x: os.path.splitext(x)[0]);

results_df_final = results_df_cleaned[['subject/task', 'stress_probability']];

output_csv_path = '/content/drive/MyDrive/speechtexttranscriptionpredicted.csv';
results_df_final.to_csv(output_csv_path, index=False);

print(f"saved to '{output_csv_path}'.");
print(results_df_final.head());

Results successfully saved to '/content/drive/MyDrive/speechtexttranscriptionpredicted.csv'.
First 5 rows of the saved DataFrame:
     subject/task  stress_probability
0  t6v9_Counting2            0.008007
1  t6v9_Counting3            0.005339
2     t6v9_Stroop            0.003146
3       t6v9_Math            0.004755
4  t6v9_Counting1            0.008945


In [ ]:
import pandas as pd


validation_df = pd.read_csv('/content/drive/MyDrive/WuHaoAllenCentad/stressidtest2.csv');
print("First 5 rows of validation_df:");
print(validation_df.head());


predictions_df = pd.read_csv('/content/drive/MyDrive/speechtexttranscriptionpredicted.csv');
print("First 5 rows of predictions_df:");
print(predictions_df.head());

Validation dataset loaded. Shape: (70, 7)
First 5 rows of validation_df:
     subject/task  binary-stress  affect3-class subject  isuseful  \
0  2ea4_Counting1              1              2    2ea4      True   
1  2ea4_Counting2              1              2    2ea4      True   
2  2ea4_Counting3              1              2    2ea4      True   
3       2ea4_Math              1              2    2ea4      True   
4    2ea4_Reading              0              0    2ea4      True   

                                                path  continuouslabel  
0  /content/drive/MyDrive/WuHaoAllenCentad/stress...                7  
1  /content/drive/MyDrive/WuHaoAllenCentad/stress...                6  
2  /content/drive/MyDrive/WuHaoAllenCentad/stress...                6  
3  /content/drive/MyDrive/WuHaoAllenCentad/stress...                6  
4  /content/drive/MyDrive/WuHaoAllenCentad/stress...                4  

Predictions dataset loaded. Shape: (371, 2)
First 5 rows of predictions_df:
   

In [ ]:
#rename columns
validation_df = validation_df.rename(columns={'subject/task': 'subject_task'});
print("validation_df columns after renaming:");
print(validation_df.columns);

predictions_df['predicted stressed probability'] = pd.to_numeric(predictions_df['predicted stressed probability'], errors='coerce');
print("\npredictions_df info after converting 'predicted stressed probability' to numeric:");
predictions_df.info();

print(validation_df.head());
print(predictions_df.head());

In [ ]:
merged_df = pd.merge(validation_df, predictions_df, on='subject_task', how='inner');

print(merged_df.head());

Merged DataFrame shape: (70, 8)
First 5 rows of merged_df:
     subject_task  binary-stress  affect3-class subject  isuseful  \
0  2ea4_Counting1              1              2    2ea4      True   
1  2ea4_Counting2              1              2    2ea4      True   
2  2ea4_Counting3              1              2    2ea4      True   
3       2ea4_Math              1              2    2ea4      True   
4    2ea4_Reading              0              0    2ea4      True   

                                                path  continuouslabel  \
0  /content/drive/MyDrive/WuHaoAllenCentad/stress...                7   
1  /content/drive/MyDrive/WuHaoAllenCentad/stress...                6   
2  /content/drive/MyDrive/WuHaoAllenCentad/stress...                6   
3  /content/drive/MyDrive/WuHaoAllenCentad/stress...                6   
4  /content/drive/MyDrive/WuHaoAllenCentad/stress...                4   

   predicted stressed probability  
0                        0.010016  
1              

In [ ]:
# get labels
true_labels = merged_df['binary-stress'];
print(true_labels.head());


predicted_probabilities = merged_df['predicted stressed probability'];
print(predicted_probabilities.head());

Extracted true_labels. First 5 values:
0    1
1    1
2    1
3    1
4    0
Name: binary-stress, dtype: int64

Extracted predicted_probabilities. First 5 values:
0    0.010016
1    0.012527
2    0.004846
3    0.002763
4    0.001794
Name: predicted stressed probability, dtype: float64


In [ ]:
import pandas as pd


predicted_labels = (predicted_probabilities >= 0.5).astype(int);

print(predicted_labels.head());

Created predicted_labels. First 5 values:
0    0
1    0
2    0
3    0
4    0
Name: predicted stressed probability, dtype: int64
